In [1]:
#!pip install pyfim
# !pip install pandas
# !pip install matplotlib
# !pip install unlzw
# !pip install pyfim

In [2]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)

In [3]:
# Load the iris dataset
iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
iris = pd.read_csv('iris.data', header = None)
iris.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
iris.to_csv("iris.csv", index = None, header = True)
#iris = pd.read_csv('iris.csv',index= None, header = True)

#Load the waveform dataset #CART decision tree algorithm: 72% (Information from the .names file)
with open('waveform.data.Z', 'rb') as fh:
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)
data = uncompressed_data.decode('utf-8').splitlines()
with open("wave.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+',line))
wave = pd.read_csv('wave.csv', header = None)
wave.to_csv("wave.csv", index = None, header = True)
wave_labels = []
for i in range(21):
  wave_labels.append('x'+str(i))
wave_labels.append('class')
wave.columns=wave_labels
wave.to_csv("wave.csv", index = None, header = True)

#Load the breast cancer dataset
breastCancer = pd.read_csv("breast-cancer.data")
breastCancer.columns=["class","age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]
breastCancer.to_csv("breastCancer.csv", index = None, header = True)
#wave.to_csv("breastCancer.csv", index = None, header = True)


#Load the german dataset
german = pd.read_csv('german.data', header = None)
german.head()
german_columns=["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker","goodBad"]
german = german[0].str.split(" ", n = 20, expand = True)
german.columns = german_columns
german.to_csv("german.csv", index = None, header = True)
#german.columns = german_columns

#Load the heart dataset
heart = pd.read_csv('heart.dat', header = None)
heart = heart[0].str.split(" ", n = 13, expand = True)
heart.columns=["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal","class"]
heart_columns= ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]
heart.to_csv("heart.csv", index = None, header = True)


#Load the kaggle dataset
#stroke
stroke = pd.read_csv("healthcare-dataset-stroke-data.csv", sep=";", index_col=0)

#Heart Attack
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv", sep=";", index_col=0)

In [37]:
wave.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,class
0,-1.23,-1.56,-1.75,-0.28,0.60,2.22,0.85,0.21,-0.20,0.89,...,2.89,7.75,4.59,3.15,5.12,3.32,1.20,0.24,-0.56,2
1,-0.69,2.43,0.61,2.08,2.30,3.25,5.52,4.55,2.97,2.22,...,1.24,1.89,1.88,-1.34,0.83,1.41,1.78,0.60,2.42,1
2,-0.12,-0.94,1.29,2.59,2.42,3.55,4.94,3.25,1.90,2.07,...,2.50,0.12,1.41,2.78,0.64,0.62,-0.01,-0.79,-0.12,0
3,0.86,0.29,2.19,-0.02,1.13,2.51,2.37,5.45,5.45,4.84,...,2.58,1.40,1.24,1.41,1.07,-1.43,2.84,-1.18,1.12,1
4,1.16,0.37,0.40,-0.59,2.66,1.00,2.69,4.06,5.34,3.53,...,4.30,1.84,1.73,0.21,-0.18,0.13,-0.21,-0.80,-0.68,1


Discretising the rules

In [4]:
import Orange

In [5]:
from yarc import CBA
from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
import pandas as pd

In [6]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

In [27]:
def runCBA(X,y,target):
    X = discretiseRule(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    txns = TransactionDB.from_DataFrame(train, target=target) #../../
    txnstest = TransactionDB.from_DataFrame(test, target=target) #../../

    cba = CBA()
    cba.fit(txns)
    cba.target_class
    cba.pre.rules
    
    cbaTrainAcc=cba.rule_model_accuracy(txns)
    cbaTestAcc=cba.rule_model_accuracy(txnstest)
    # get the best association rules
    rules = top_rules(txns.string_representation)

    # convert them to class association rules
    cars = CARlist(rules)

    predictor = M2Classi(cars, txns).build()

    accuracy = predictor.test_transactions(txnstest)
    #print(cba.pre.rules)
    print("CBA Train Accuracy:",str(cbaTrainAcc))
    print("CBA Test Accuracy:",str(cbaTestAcc))
    print("Top K Test Accuracy:",str(accuracy))

In [28]:
from sklearn.model_selection import train_test_split
import pandas as pd
from Orange.data.pandas_compat import table_from_frame,table_to_frame

read_file = pd.read_csv ('iris.csv') #discretised iris from their database #currently its the pure one without discretising

X=read_file[["sepallength","petalwidth","sepalwidth","petallength"]]
y=read_file[["class"]]

X = discretiseRule(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

iris_cba = CBA()

In [29]:
iris_cba.fit(txns)
iris_cba.target_class
iris_cba.predict_probability(txns)
iris_cba.pre.rules
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)
iris_cba.predict_matched_rules(txns)

[CAR {Feature 3=2.45 - 4.85,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.28 conf: 1.00 len: 3, id: 215490,
 CAR {Feature 3=≥ 4.85,Feature 1=≥ 6.25,Feature 4=≥ 1.55} => {class=Iris-virginica} sup: 0.23 conf: 0.97 len: 4, id: 215499,
 CAR {Feature 3=≥ 4.85,Feature 4=≥ 1.55} => {class=Iris-virginica} sup: 0.29 conf: 0.95 len: 3, id: 215523,
 CAR {Feature 3=2.45 - 4.85,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.28 conf: 1.00 len: 3, id: 215490,
 CAR {Feature 3=< 2.45} => {class=Iris-setosa} sup: 0.33 conf: 1.00 len: 2, id: 215512,
 CAR {Feature 3=< 2.45} => {class=Iris-setosa} sup: 0.33 conf: 1.00 len: 2, id: 215512,
 CAR {Feature 3=≥ 4.85,Feature 1=≥ 6.25,Feature 4=≥ 1.55} => {class=Iris-virginica} sup: 0.23 conf: 0.97 len: 4, id: 215499,
 CAR {Feature 3=< 2.45} => {class=Iris-setosa} sup: 0.33 conf: 1.00 len: 2, id: 215512,
 CAR {Feature 3=2.45 - 4.85,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.28 conf: 1.00 len: 3, id: 215490,
 CAR {Feature 4=0.8 - 1.

In [30]:
cbaTrainAcc=iris_cba.rule_model_accuracy(txns)
cbaTestAcc=iris_cba.rule_model_accuracy(txnstest)
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print("CBA Train Accuracy:",str(cbaTrainAcc))
print("CBA Test Accuracy:",str(cbaTestAcc))
print("Top K Test Accuracy:",str(accuracy))

CBA Train Accuracy: 0.9583333333333334
CBA Test Accuracy: 0.9333333333333333
Top K Test Accuracy: 0.9333333333333333


Heart

In [34]:
read_file = pd.read_csv ('heart.csv')
X=read_file[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]

runCBA(X,y,"class")

CBA Train Accuracy: 0.9027777777777778
CBA Test Accuracy: 0.7592592592592593
Top K Test Accuracy: 0.7777777777777778


Breast Cancer

In [35]:
#error: cannot convert string to float
read_file = pd.read_csv ('breastCancer.csv')
X=read_file[["age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
y=read_file[["class"]]
runCBA(X,y,"class")

ValueError: could not convert string to float: '40-49'

In [ ]:
#error: cannot convert string to float
read_file = pd.read_csv ('breastCancer.csv')
X=read_file[["age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
y=read_file[["class"]]
runCBA(X,y,"class")

German

In [36]:
#error: cannot convert string to float
read_file = pd.read_csv ('german.csv')
X=read_file[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
y=read_file[["goodBad"]]
runCBA(X,y,"goodBad")

ValueError: could not convert string to float: 'A11'

In [13]:
cba.predict_probability(txns)

[1.0,
 0.9714285714285714,
 0.9696969696969697,
 0.9696969696969697,
 0.9411764705882353,
 1.0,
 0.9714285714285714,
 1.0,
 1.0,
 1.0,
 0.9705882352941176,
 1.0,
 0.9583333333333334,
 1.0,
 0.92,
 0.975609756097561,
 1.0,
 0.9629629629629629,
 0.9666666666666667,
 0.9354838709677419,
 1.0,
 1.0,
 1.0,
 0.9629629629629629,
 1.0,
 1.0,
 0.9705882352941176,
 0.9285714285714286,
 1.0,
 0.9772727272727273,
 1.0,
 0.9,
 0.9565217391304348,
 0.4444444444444444,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9375,
 0.9696969696969697,
 0.9583333333333334,
 0.9696969696969697,
 1.0,
 0.9714285714285714,
 0.9705882352941176,
 1.0,
 0.9705882352941176,
 1.0,
 1.0,
 0.9310344827586207,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9666666666666667,
 0.96,
 1.0,
 0.92,
 1.0,
 1.0,
 0.9411764705882353,
 1.0,
 0.9629629629629629,
 0.9655172413793104,
 0.9565217391304348,
 1.0,
 1.0,
 0.9705882352941176,
 0.9444444444444444,
 0.9705882352941176,
 1.0,
 1.0,
 1.0,
 1.0,
 0.44

In [14]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

CAR {} => {class=default_class1} sup: 0.10 conf: 0.20 len: 1, id: 44535

In [15]:
cba.rule_model_accuracy(txns)

0.9027777777777778

In [16]:
cba.rule_model_accuracy(txnstest)

0.7592592592592593

In [17]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)


In [18]:
print(accuracy)

0.7962962962962963


In [19]:
read_file = pd.read_csv ('heart.csv')
X=read_file[ ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

cba = CBA()

In [20]:
cba.fit(txns)
cba.target_class
cba.pre.rules

[CAR {slopePeak=1.0,resting electrocariographic results=0.0,thal=3.0} => {class=1.0} sup: 0.18 conf: 1.00 len: 4, id: 65372,
 CAR {sex=0.0,exercise induced angina=0.0,numMajorVessels=0.0} => {class=1.0} sup: 0.16 conf: 1.00 len: 4, id: 65243,
 CAR {resting blood sugar >120mg/dl=0.0,slopePeak=1.0,sex=0.0} => {class=1.0} sup: 0.13 conf: 1.00 len: 4, id: 65219,
 CAR {resting electrocariographic results=0.0,sex=0.0,exercise induced angina=0.0} => {class=1.0} sup: 0.13 conf: 1.00 len: 4, id: 65229,
 CAR {slopePeak=1.0,chest pain type=3.0,numMajorVessels=0.0} => {class=1.0} sup: 0.10 conf: 1.00 len: 4, id: 65138,
 CAR {slopePeak=1.0,thal=3.0,numMajorVessels=0.0} => {class=1.0} sup: 0.22 conf: 0.98 len: 4, id: 65396,
 CAR {slopePeak=1.0,sex=0.0} => {class=1.0} sup: 0.16 conf: 0.97 len: 3, id: 65220,
 CAR {chest pain type=4.0,slopePeak=2.0,thal=7.0} => {class=2.0} sup: 0.15 conf: 0.97 len: 4, id: 65292,
 CAR {chest pain type=4.0,thal=7.0,resting electrocariographic results=2.0} => {class=2.0} 

In [21]:
cba.predict_probability(txns)

[1.0,
 0.9714285714285714,
 0.9696969696969697,
 0.9696969696969697,
 0.9411764705882353,
 1.0,
 0.96,
 1.0,
 0.9393939393939394,
 1.0,
 0.9705882352941176,
 0.9230769230769231,
 0.92,
 1.0,
 0.92,
 0.94,
 1.0,
 0.9565217391304348,
 0.9285714285714286,
 0.9354838709677419,
 0.9705882352941176,
 1.0,
 0.9705882352941176,
 0.9565217391304348,
 1.0,
 1.0,
 0.9705882352941176,
 0.8823529411764706,
 1.0,
 0.9565217391304348,
 1.0,
 0.9,
 0.9565217391304348,
 0.4444444444444444,
 1.0,
 0.9705882352941176,
 1.0,
 0.9444444444444444,
 0.9705882352941176,
 0.9795918367346939,
 1.0,
 1.0,
 0.9705882352941176,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9705882352941176,
 0.8787878787878788,
 0.9696969696969697,
 0.92,
 0.9696969696969697,
 1.0,
 0.9714285714285714,
 0.9705882352941176,
 0.9795918367346939,
 0.9705882352941176,
 1.0,
 0.8863636363636364,
 0.4444444444444444,
 1.0,
 0.9705882352941176,
 1.0,
 1.0,
 0.9285714285714286,
 0.9,
 0.9696969696969697,
 0.92,
 1.0,
 1.0,
 0.9411764705882353,
 0.9705

In [22]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

CAR {} => {class=default_class1} sup: 0.10 conf: 0.20 len: 1, id: 65511

In [23]:
cba.rule_model_accuracy(txns)

0.8888888888888888

In [24]:
cba.rule_model_accuracy(txnstest)

0.7592592592592593

In [25]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print(accuracy)

0.6666666666666666


In [26]:
pip install pyarc

Note: you may need to restart the kernel to use updated packages.
